In [4]:
import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import cartopy.crs as ccrs
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import matplotlib.patches as patches
import cmaps
import cartopy.feature as cfeature
from datetime import datetime,timedelta
from matplotlib.offsetbox import AnchoredText#关于修改text位置的
import cv2
from pygac import get_reader_class
import ast
from Five_utiles4 import *
from tqdm import tqdm
proj =ccrs.NorthPolarStereo(central_longitude=17.5)
from sklearn.neighbors import BallTree
from shapely.geometry import Point, Polygon,LineString
#经过实验，按经纬度插值的清晰度小于先转化为实际地理坐标系再插值的清晰度
import concurrent.futures
import pyproj
import matplotlib.style as mplstyle
from matplotlib.colors import BoundaryNorm
from datetime import datetime

In [1]:
def withdraw_info(filename):
    #根据ASCAT文件名提取信息
    info_list = filename.split('_')
    year = int(info_list[1][0:4])
    month = int(info_list[1][4:6])
    day = int(info_list[1][6:8])
    start_hour = int(info_list[2][0:2])
    start_minute = int(info_list[2][2:4])
    start_time = pd.Timestamp(year=year,month=month,day=day,hour=start_hour,minute=start_minute)
    satellite_name = info_list[3]
    return start_time,satellite_name

In [ ]:
def geo2lonlat(central_lon,central_lat,distance_x,distance_y):
    #位移不变的经纬度转换
    earth_radius = 6370856#m
    lats = distance_y/(2*np.pi*earth_radius)*360+central_lat
    lon_diff = distance_x/(2*np.pi*earth_radius*np.cos(central_lat/180*np.pi))*360
    return central_lon+lon_diff,lats

In [3]:
#风速向北吹为0°,顺时针旋转
def angel2uv(wind_dir,wind_speed):
    rad = np.radians(wind_dir)
    u = wind_speed * np.sin(rad)
    v = wind_speed * np.cos(rad)
    return u,v

In [6]:
cloud_infos = pd.read_excel(r'D:\Desktop\final_dataset\cloud_infos.xlsx').drop(columns=['Unnamed: 0'])
cloud_infos

,label,x1,y1,x2,y2,x3,y3,x4,y4,x0,...,satellite_name,mean_time,cloud_lon,cloud_lat,cloud_time,width,length,l_search,s_search,step
0,comma,635.500,563.625,712.706,366.707,349.473,224.295,272.268,421.213,433.000,...,NOAA-15,2001-02-21 07:44:00,9.560060,71.458664,2001-02-21 07:44:00,423.024657,780.304206,514.393408,265.911690,1
1,spiral,649.875,560.500,701.616,353.409,319.580,257.959,267.839,465.049,422.375,...,NOAA-15,2001-02-21 10:14:00,8.714921,71.551280,2001-02-21 10:14:00,426.913637,787.559264,526.875217,260.683784,2
2,spiral,387.022,665.720,564.673,591.963,448.268,311.592,270.617,385.349,385.500,...,NOAA-15,2001-02-22 09:52:00,-6.737806,67.798081,2001-02-22 09:53:00,384.707540,607.150794,443.729795,163.420999,3
3,spiral,282.612,355.957,294.491,548.038,628.625,527.375,616.746,335.294,384.875,...,NOAA-15,2001-02-22 18:57:00,-7.410107,63.935663,2001-02-22 18:57:00,384.895940,669.544590,474.842512,194.702078,4
4,comma,421.902,505.452,474.250,485.500,423.416,352.124,371.068,372.076,391.125,...,NOAA-15,2001-02-23 11:07:00,8.472583,76.792719,2001-02-23 11:06:00,112.042767,285.469809,221.566913,63.902896,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16413,comma,237.998,492.552,363.199,568.958,503.000,339.875,377.798,263.469,399.875,...,Metop-B,2023-11-06 13:36:00,43.621866,80.913697,2023-11-06 13:36:00,293.347352,536.742251,364.215308,172.527501,10
16414,comma,237.672,537.978,355.699,599.399,489.373,342.529,371.346,281.108,398.824,...,Metop-C,2023-11-06 14:29:00,44.497828,81.071083,2023-11-06 14:29:00,266.104581,579.140519,423.474399,155.666120,11
16415,comma,243.955,525.834,359.803,589.770,502.202,331.750,386.354,267.814,402.827,...,NOAA-19,2023-11-06 14:48:00,44.962854,81.118730,2023-11-06 14:48:00,264.639915,589.412574,408.015448,181.397126,12
16416,comma,287.614,501.793,405.229,539.517,474.875,322.375,357.260,284.651,398.000,...,NOAA-18,2023-11-06 15:01:00,45.146491,81.219906,2023-11-06 15:01:00,247.033507,456.075491,309.134777,146.940714,13


In [7]:
def near_judge(chosed_cloud_info,chosed_wind_urlinfo):
    #匹配ASCAT和quickscat数据
    result = [] 
    chosed_time,chosed_lon,chosed_lat = chosed_cloud_info[['cloud_time','cloud_lon','cloud_lat']]
    chosed_time = pd.Timestamp(chosed_time)
    wind_data = xr.open_dataset(chosed_wind_urlinfo.path)
    #将风速经纬度数据统一转化为-180至180°
    lons,lats,times = wind_data.lon.values,wind_data.lat.values,wind_data.time
    lons = np.where(lons>180,lons-360,lons)
    disses = geodistance(chosed_lon,chosed_lat,lons,lats)
    near_num= np.nansum(disses<50)
    #当小于50KM的风速数据大于20个，且平均扫描时间在当前时间内，保留信息
    if near_num>0:
        if chosed_wind_urlinfo.satellite=='QS':
            scantimes = times.values[(disses<50).sum(axis=1)>0]
            wind_speed =  wind_data.retrieved_wind_speed.values
            mean_time = pd.Series(scantimes).mean()
        else:
            scantimes = times.values[(disses<50)]
            scantimes = scantimes[~np.isnan(scantimes)]
            wind_speed = wind_data.wind_speed.values
            mean_time = pd.Series(scantimes).mean()
        wind_near_num= np.nansum(~np.isnan(wind_speed[disses<50]))
        #当风速数据不够时，不保留信息
        if wind_near_num<10:
            return result
        #当扫描时间在30分钟外，不保留信息
        if mean_time>chosed_time:
            #三十分钟内
            minute_diff = (mean_time-chosed_time).total_seconds()/60
        else:
            minute_diff = (chosed_time-mean_time).total_seconds()/60
        if minute_diff<30:
            chosed_cloud_info['wind_path'] = chosed_wind_urlinfo.path
            chosed_cloud_info['wind_mean_time'] = pd.to_datetime( mean_time.strftime('%Y%m%d_%H%M'),format='%Y%m%d_%H%M')
            chosed_cloud_info['wind_satellite'] =  chosed_wind_urlinfo.satellite
            chosed_cloud_info['wind_match_num']  =   wind_near_num
            chosed_cloud_info['minutes_diff']  =   minute_diff
            #长搜索半径内75分位数风场和短KM内10m最大风场
            chosed_cloud_info['env_advection_wind']  =  np.nanpercentile(wind_speed[(disses<chosed_cloud_info.l_search)],75)
            #短搜索半径内10m风场
            in_r_bool = (disses<chosed_cloud_info.s_search)
            in_r_lons,in_r_lats,in_r_wind = lons[in_r_bool],lats[in_r_bool],wind_speed[in_r_bool]
            chosed_cloud_info['max_wind']  =  np.nanmax(in_r_wind)
            max_wind_index = np.nanargmax(in_r_wind)
            chosed_cloud_info['wind_lon']  =  in_r_lons[max_wind_index]
            chosed_cloud_info['wind_lat']  = in_r_lats[max_wind_index]
            result.append(pd.DataFrame(chosed_cloud_info).T)
    return result

风速计信息提取

In [8]:
#ASCAT信息提取
ASCAT_folder = r"Y:\ASCAT_data"
ASCAT_filenames = os.listdir(ASCAT_folder)
ASCAT_info = pd.DataFrame(ASCAT_filenames,columns=['filename'])
infos = np.array([withdraw_info(os.path.basename(i)) for i in ASCAT_filenames])
ASCAT_info['start_time'] = infos[:,0]
ASCAT_info['satellite'] = 'AS'
ASCAT_info['month'] = ASCAT_info['start_time'].apply(lambda x:x.month)
ASCAT_info['integral_time'] = ASCAT_info['start_time'].apply(lambda x:x-timedelta(seconds=x.minute*60)+timedelta(seconds=60*60)*round(x.minute/60))#将分钟去掉
ASCAT_info['path'] = [os.path.join(ASCAT_folder,i) for i in ASCAT_filenames]
#qs信息提取
qs_folder = r"Y:\qs_data"
qs_filenames = os.listdir(qs_folder)
qs_info = pd.DataFrame(qs_filenames,columns=['filename'])
qs_info['start_time'] =  [datetime.strptime(os.path.basename(i).split('_')[-1].split('.')[0], "%Y%m%d%H%M") for i in qs_filenames]
qs_info['satellite'] = 'QS'
qs_info['month'] = qs_info['start_time'].apply(lambda x:x.month)
qs_info['integral_time'] = qs_info['start_time'].apply(lambda x:x-timedelta(seconds=x.minute*60)+timedelta(seconds=60*60)*round(x.minute/60))#将分钟去掉
qs_info['path'] = [os.path.join(qs_folder,i) for i in qs_filenames]
wind_info = pd.concat([qs_info,ASCAT_info],axis=0).reset_index(drop=True)
wind_info

,filename,start_time,satellite,month,integral_time,path
0,qs_l2b_02784_v4.1_199912312316.nc4,1999-12-31 23:16:00,QS,12,1999-12-31 23:00:00,Y:\qs_data\qs_l2b_02784_v4.1_199912312316.nc4
1,qs_l2b_02785_v4.1_200001010057.nc4,2000-01-01 00:57:00,QS,1,2000-01-01 01:00:00,Y:\qs_data\qs_l2b_02785_v4.1_200001010057.nc4
2,qs_l2b_02786_v4.1_200001010238.nc4,2000-01-01 02:38:00,QS,1,2000-01-01 03:00:00,Y:\qs_data\qs_l2b_02786_v4.1_200001010238.nc4
3,qs_l2b_02787_v4.1_200001010419.nc4,2000-01-01 04:19:00,QS,1,2000-01-01 04:00:00,Y:\qs_data\qs_l2b_02787_v4.1_200001010419.nc4
4,qs_l2b_02788_v4.1_200001010600.nc4,2000-01-01 06:00:00,QS,1,2000-01-01 06:00:00,Y:\qs_data\qs_l2b_02788_v4.1_200001010600.nc4
...,...,...,...,...,...,...
89352,ascat_20250101_202400_metopc_31935_eps_o_coa_3...,2025-01-01 20:24:00,AS,1,2025-01-01 20:00:00,Y:\ASCAT_data\ascat_20250101_202400_metopc_319...
89353,ascat_20250101_210600_metopb_63778_eps_o_coa_3...,2025-01-01 21:06:00,AS,1,2025-01-01 21:00:00,Y:\ASCAT_data\ascat_20250101_210600_metopb_637...
89354,ascat_20250101_220600_metopc_31936_eps_o_coa_3...,2025-01-01 22:06:00,AS,1,2025-01-01 22:00:00,Y:\ASCAT_data\ascat_20250101_220600_metopc_319...
89355,ascat_20250101_224800_metopb_63779_eps_o_coa_3...,2025-01-01 22:48:00,AS,1,2025-01-01 23:00:00,Y:\ASCAT_data\ascat_20250101_224800_metopb_637...


In [9]:
#匹配风速
results = []
for i in tqdm(range(len(cloud_infos))):
    chosed_cloud_info = cloud_infos.loc[i].copy()
    chosed_time,chosed_lon,chosed_lat = chosed_cloud_info[['cloud_time','cloud_lon','cloud_lat']]
    chosed_time = pd.Timestamp(chosed_time)
    #这里把时间定为两个小时，因为从文件名只能得到相隔较久的起始时间
    candidate_wind_info = wind_info.loc[np.logical_and(wind_info.start_time<chosed_time,
                                            wind_info.start_time>(chosed_time-timedelta(hours=4)))]
    #对每个可能轨迹进行匹配
    for j in range(len(candidate_wind_info)):
        chosed_wind_urlinfo = candidate_wind_info.iloc[j]
        temp_result = near_judge(chosed_cloud_info,chosed_wind_urlinfo)
        if len(temp_result):
            results+=temp_result
len(results)

100%|██████████| 16418/16418 [1:15:49<00:00,  3.61it/s]


4607

In [10]:
wind = pd.concat(results,axis=0).infer_objects()
wind

,label,x1,y1,x2,y2,x3,y3,x4,y4,x0,...,step,wind_path,wind_mean_time,wind_satellite,wind_match_num,minutes_diff,env_advection_wind,max_wind,wind_lon,wind_lat
17,comma,404.250,537.375,469.523,424.518,430.182,401.765,364.910,514.622,422.375,...,4,Y:\qs_data\qs_l2b_08812_v4.1_200102262225.nc4,2001-02-26 23:13:00,QS,37,27.705480,11.757554,21.849976,49.859161,73.061317
18,comma,420.500,529.250,482.895,421.369,441.810,397.607,379.415,505.488,436.750,...,5,Y:\qs_data\qs_l2b_08813_v4.1_200102270006.nc4,2001-02-27 00:52:00,QS,18,29.010719,10.661333,25.607101,49.305264,73.045586
56,comma,156.214,585.108,245.647,674.411,456.750,463.000,367.317,373.697,315.500,...,1,Y:\qs_data\qs_l2b_08856_v4.1_200103020031.nc4,2001-03-02 01:18:00,QS,54,27.041050,13.384654,18.769762,34.763649,74.291656
57,comma,156.214,585.108,260.685,689.426,508.000,441.750,403.529,337.432,343.000,...,2,Y:\qs_data\qs_l2b_08857_v4.1_200103020212.nc4,2001-03-02 02:58:00,QS,51,26.341851,13.782904,24.105286,39.293949,75.041092
68,comma,608.000,546.750,664.866,378.054,327.042,264.177,270.176,432.872,381.254,...,3,Y:\qs_data\qs_l2b_09108_v4.1_200103191650.nc4,2001-03-19 17:45:00,QS,49,6.322367,14.864102,23.490767,4.786680,74.866844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16382,spiral,309.342,486.047,416.750,518.000,456.349,384.890,348.941,352.937,381.125,...,6,Y:\ASCAT_data\ascat_20230331_140900_metopb_546...,2023-03-31 14:32:00,AS,51,1.466667,11.875000,13.910000,5.573680,76.337540
16383,spiral,304.610,463.455,400.798,502.491,449.557,382.343,353.370,343.307,378.625,...,7,Y:\ASCAT_data\ascat_20230331_150000_metopc_228...,2023-03-31 15:24:00,AS,52,6.200000,12.230000,14.260000,5.857460,76.558823
16384,spiral,398.867,502.990,452.020,386.234,356.990,342.972,303.837,459.728,379.250,...,8,Y:\ASCAT_data\ascat_20230331_150000_metopc_228...,2023-03-31 15:24:00,AS,51,19.200000,12.157500,14.260000,5.857460,76.558823
16391,spiral,533.000,362.375,464.708,265.158,305.644,376.896,373.936,474.113,406.125,...,3,Y:\ASCAT_data\ascat_20230331_113900_metopc_228...,2023-03-31 12:09:00,AS,51,0.600000,15.150000,17.139999,-32.067810,61.254768


In [36]:
wind = pd.read_excel(r'D:\Desktop\final_dataset\cloud_wind.xlsx').drop(columns=['Unnamed: 0'])
purewind = wind.dropna(how='any')
other = wind.loc[~np.isin(wind.index,purewind.index)]
best_wind_index = purewind.loc[purewind.groupby('filename')['wind_match_num'].idxmax()]
chosed_index  = np.concatenate([best_wind_index.index,other.index])
wind.loc[np.sort(chosed_index)].reset_index(drop=True).to_excel(r'D:\Desktop\final_dataset\cloud_wind.xlsx')

In [ ]:
cloud_infos = pd.read_excel(r'D:\Desktop\final_dataset\cloud_wind.xlsx').drop(columns=['Unnamed: 0'])

In [11]:
use_cols = ['wind_mean_time', 'wind_satellite', 'wind_match_num', 'minutes_diff','env_advection_wind', 'max_wind', 'wind_lon', 'wind_lat','wind_path','filename']
cloud_infos = pd.merge(cloud_infos,wind[use_cols],on='filename',how='left')
cloud_infos.to_excel(r'D:\Desktop\final_dataset\cloud_wind.xlsx')